In [5]:
from TAI.source import Alpaca
ap = Alpaca()

# get_option_md_client = OptionHistoricalDataClient(self.api_key, self.api_secret)
# 




In [ ]:
req = OptionLatestTradeRequest(symbol_or_symbols=symbol_or_symbols)
option_md_client.get_option_latest_trade(req)

In [9]:
ap.get_latest_trade(symbol_or_symbols='SPY240920C00325000',asset='option')

{'SPY240920C00325000': {   'conditions': 'f',
     'exchange': 'W',
     'id': None,
     'price': 202.95,
     'size': 1.0,
     'symbol': 'SPY240920C00325000',
     'tape': None,
     'timestamp': datetime.datetime(2024, 8, 6, 20, 8, 46, 480139, tzinfo=TzInfo(UTC))}}

In [7]:
df_chain = ap.get_option_chain(underlying_symbol='SPY',
                    feed=None,
                    type=None,
                    strike_price_gte=None, #550.0,
                    strike_price_lte=None, #560.0,
                    expiration_date='2024-09-20',
                    expiration_date_gte=None,
                    expiration_date_lte=None,
                    root_symbol=None,
                    raw=False,
                    chain_table=False)
df_chain
# [['strike_price','ask_price','bid_price','type']]


# df_chain.loc[df_chain['strike_price']==573.0]a

,symbol,delta,gamma,rho,theta,vega,implied_volatility,ask_price,ask_size,bid_price,...,open_interest_date,root_symbol,size_y,status,strike_price,style,tradable,type,underlying_asset_id,underlying_symbol
0,SPY240920C00215000,NaN,NaN,NaN,NaN,NaN,NaN,332.15,50.0,331.05,...,2024-09-06,SPY,100,active,215.0,american,True,call,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,SPY
1,SPY240920C00430000,0.9863,0.0006,0.1501,-0.1391,0.0357,0.5669,111.75,10.0,111.17,...,2024-09-06,SPY,100,active,430.0,american,True,call,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,SPY
2,SPY240920C00325000,NaN,NaN,NaN,NaN,NaN,NaN,216.28,50.0,213.15,...,2024-09-06,SPY,100,active,325.0,american,True,call,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,SPY
3,SPY240920C00380000,NaN,NaN,NaN,NaN,NaN,NaN,163.48,250.0,162.52,...,2024-09-06,SPY,100,active,380.0,american,True,call,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,SPY
4,SPY240920C00290000,NaN,NaN,NaN,NaN,NaN,NaN,250.94,50.0,249.90,...,2024-09-06,SPY,100,active,290.0,american,True,call,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,SPY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,SPY240920P00544000,-0.5384,0.0178,-0.1073,-0.2964,0.4050,0.2185,10.30,57.0,10.25,...,2024-09-06,SPY,100,active,544.0,american,True,put,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,SPY
366,SPY240920P00546000,-0.5743,0.0178,-0.1146,-0.2861,0.3998,0.2165,11.37,20.0,11.31,...,2024-09-06,SPY,100,active,546.0,american,True,put,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,SPY
367,SPY240920P00558000,-0.7691,0.0141,-0.1549,-0.1897,0.3104,0.2120,20.04,300.0,18.83,...,2024-09-06,SPY,100,active,558.0,american,True,put,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,SPY
368,SPY240920P00579000,-0.8788,0.0066,-0.1831,-0.1601,0.2054,0.2973,39.65,250.0,38.98,...,None,SPY,100,active,579.0,american,True,put,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,SPY


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from TAI.source import Alpaca

class OptionBet:
    def __init__(self, ticker, expiry_date):
        self.ap = Alpaca()
    
        self.ticker = ticker
        self.expiry_date = expiry_date
        
        self.today = date.today()
        self.today_str = self.today.strftime("%Y-%m-%d")
        self.df_raw = self.ap.get_stock_historical(symbol_or_symbols=self.ticker, 
                                                lookback_period=720, 
                                                timeframe='Day', 
                                                end=None, 
                                                currency='USD', 
                                                limit=None, 
                                                adjustment='all', 
                                                feed=None, 
                                                asof=None, 
                                                sort='asc', 
                                                raw=False)
        
        self.df = self.df_raw[['timestamp','open','high','low','close','volume']].set_index('timestamp')
        self.current_price = self.ap.get_latest_trade(self.ticker).get(self.ticker).price
        self.open=self.df.tail(1)['open'][0]

    def nearest_value(self, input_list, find_value):
        difference = lambda input_list: abs(input_list - find_value)
        return min(input_list, key=difference)

    def weekdays_calculator(self, end_str):
        end = datetime.strptime(end_str, '%Y-%m-%d').date()
        return np.busday_count(self.today, end)

    def perc_change(self, df, horizon):
        df['return_perc'] = df.pct_change(periods=horizon - 1, fill_method='ffill').round(decimals=4)['close']
        return df.dropna()

    def describe_perc_change(self):
        horizon = self.weekdays_calculator(self.expiry_date)
        df_perc_change = self.perc_change(self.df, horizon)[['close', 'return_perc']]
        select_price = self.current_price
        describe = df_perc_change.describe(percentiles=[.001, .01, .05, .1, .15, .25, .5, .75, .85, .9, .95, .99, .999])
        describe['current_price'] = select_price
        describe['projected_price'] = select_price * (1 + describe['return_perc'])
        describe['percentage_change'] = describe['return_perc'] * 100
        describe['chose_strike'] = describe['projected_price'].astype(int)
        return describe[['current_price', 'percentage_change', 'projected_price', 'chose_strike']]


# Example usage
if __name__ == "__main__":
    ticker = 'SPY'
    chose_expiry_date = '2024-09-20'
    option_bet = OptionBet(ticker, chose_expiry_date)
    describe_df = option_bet.describe_perc_change()
    print(describe_df)
